In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import random
import re

from tqdm import tqdm
import time

import pydicom as dicom
import nibabel as nib
import SimpleITK as sitk
import monai

import torch
import torch.nn as nn
import torch.optim as optim

from monai.networks.nets import EfficientNetBN
from monai.networks.nets import ResNet
#from efficientnet_pytorch import EfficientNet

import wandb


In [2]:
SEED = 344
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # Fix the network according to random seed
    print('Finish seeding with seed {}'.format(seed))
    
seed_everything(SEED)
print('Training on device {}'.format(device))

Finish seeding with seed 344
Training on device cuda


In [3]:
dicom_tag_columns = [
    'Columns',
    'ImageOrientationPatient',
    'ImagePositionPatient',
    'InstanceNumber',
    'PatientID',
    'PatientPosition',
    'PixelSpacing',
    'RescaleIntercept',
    'RescaleSlope',
    'Rows',
    'SeriesNumber',
    'SliceThickness',
    'path',
    'WindowCenter',
    'WindowWidth'
]

train_dicom_tags = pd.read_parquet('autodl-tmp/train_dicom_tags.parquet', columns=dicom_tag_columns)
test_dicom_tags = pd.read_parquet('autodl-tmp/test_dicom_tags.parquet', columns=dicom_tag_columns)

train_series_meta = pd.read_csv('autodl-tmp/train_series_meta.csv')
test_series_meta = pd.read_csv('autodl-tmp/test_series_meta.csv')

train_csv = pd.read_csv('autodl-tmp/train.csv')

train_csv

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,9951,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3143,9960,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3144,9961,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3145,9980,1,0,1,0,1,0,0,1,0,0,0,0,1,1


In [4]:
injury_series_meta = train_series_meta.loc[train_series_meta.patient_id.isin(train_csv.loc[train_csv.any_injury == 1, "patient_id"].values)]
healthy_series_meta = train_series_meta.loc[train_series_meta.patient_id.isin(train_csv.loc[train_csv.any_injury == 0, "patient_id"].values)]

In [5]:
def raw_path_gen(patient_id, series_id, train=True):
    if(train):
        path = 'autodl-tmp/train_images_resample/'
    else:
        path = 'autodl-tmp/train_images_resample/'
    
    path += str(patient_id) + '/' + str(series_id)
    
    return path

def create_3D_scans(folder, downsample_rate=1): 
    filenames = os.listdir(folder)
    filenames = [int(filename.split('.')[0]) for filename in filenames]
    filenames = sorted(filenames)
    filenames = [str(filename) + '.dcm' for filename in filenames]
        
    volume = []
    #for filename in tqdm(filenames[::downsample_rate], position=0): 
    for filename in filenames[::downsample_rate]: 
        filepath = os.path.join(folder, filename)
        ds = dicom.dcmread(filepath)
        image = ds.pixel_array
        
        if ds.PixelRepresentation == 1:
            bit_shift = ds.BitsAllocated - ds.BitsStored
            dtype = image.dtype 
            image = (image << bit_shift).astype(dtype) >>  bit_shift
        
        # find rescale params
        if ("RescaleIntercept" in ds) and ("RescaleSlope" in ds):
            intercept = float(ds.RescaleIntercept)
            slope = float(ds.RescaleSlope)
    
        # find clipping params
        center = int(ds.WindowCenter)
        width = int(ds.WindowWidth)
        low = center - width / 2
        high = center + width / 2    
        
        
        image = (image * slope) + intercept
        image = np.clip(image, low, high)

        image = (image / np.max(image) * 255).astype(np.int16)
        image = image[::downsample_rate, ::downsample_rate]
        volume.append( image )
    
    volume = np.stack(volume, axis=0)
    return volume

def plot_image_with_seg(volume, volume_seg=[], orientation='Coronal', num_subplots=20):
    # simply copy
    if len(volume_seg) == 0:
        plot_mask = 0
    else:
        plot_mask = 1
        
    if orientation == 'Coronal':
        slices = np.linspace(0, volume.shape[2]-1, num_subplots).astype(np.int16)
        volume = volume.transpose([1, 0, 2])
        if plot_mask:
            volume_seg = volume_seg.transpose([1, 0, 2])
        
    elif orientation == 'Sagittal':
        slices = np.linspace(0, volume.shape[2]-1, num_subplots).astype(np.int16)
        volume = volume.transpose([2, 0, 1])
        if plot_mask:
            volume_seg = volume_seg.transpose([2, 0, 1])

    elif orientation == 'Axial':
        slices = np.linspace(0, volume.shape[0]-1, num_subplots).astype(np.int16)
           
    rows = np.max( [np.floor(np.sqrt(num_subplots)).astype(int) - 2, 1])
    cols = np.ceil(num_subplots/rows).astype(int)
    
    fig, ax = plt.subplots(rows, cols, figsize=(cols * 2, rows * 4))
    fig.tight_layout(h_pad=0.01, w_pad=0)
    
    ax = ax.ravel()
    for this_ax in ax:
        this_ax.axis('off')

    for counter, this_slice in enumerate( slices ):
        plt.sca(ax[counter])
        
        image = volume[this_slice, :, :]
        plt.imshow(image, cmap='gray')
        
        if plot_mask:
            mask = np.where(volume_seg[this_slice, :, :], volume_seg[this_slice, :, :], np.nan)
            plt.imshow(mask, cmap='Set1', alpha=0.5)
            
def load_nii(patient_id, series_id, root='autodl-tmp/train_images_resample/'):
    path = root + str(patient_id) + '/' + str(series_id) + '.nii.gz'
    img = sitk.ReadImage(path)
    img = sitk.GetArrayFromImage(img)
    
    # img = nib.load(path)
    # img = img.get_fdata().transpose(2, 1, 0)
    
    return img

In [6]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

ds = 2

class CTDataset(Dataset):
    def __init__(self, root='autodl-tmp/train_images_resample/', augmentation=False, meta=train_series_meta, device='cpu'):
        self.device = device
        self.series_meta = meta
        self.root = root
        self.t = monai.transforms.Compose([monai.transforms.RandZoom(prob=0.5, min_zoom=0.9, max_zoom=1.1),
                                           monai.transforms.RandRotate(range_x=3.14 / 24, prob=0.5),
                                           monai.transforms.SpatialPad(spatial_size=(352//ds, 280//ds, 280//ds), mode="edge"),
                                           monai.transforms.RandSpatialCrop(roi_size=(320//ds, 256//ds, 256//ds), random_size=False),
                                           monai.transforms.NormalizeIntensity(divisor = 400)
                                ])
        self.t_val = monai.transforms.Compose([monai.transforms.NormalizeIntensity(divisor = 400)])
        
        self.aug = augmentation
        
    def __len__(self):
        #return 1100
        return len(self.series_meta)
    
    def __getitem__(self, idx):

        patient_id, series_id = self.series_meta.loc[idx, ["patient_id", "series_id"]].astype('int')
        img_a = load_nii(patient_id, series_id, self.root).astype('float32')
        #img_t = torch.from_numpy(img_a).unsqueeze(0)
        #img_t = torch.from_numpy(img_a[::2, ::2, ::2]).unsqueeze(0)
        if(self.aug):
            img_t = self.t(np.expand_dims(img_a[::ds, ::ds, ::ds], 0))
        else:
            #img_t = torch.from_numpy(img_a[::ds, ::ds, ::ds]).unsqueeze(0)
            img_t = self.t_val(np.expand_dims(img_a[::ds, ::ds, ::ds], 0))
        label_columns = [
            'bowel_healthy',
            'bowel_injury',
            'extravasation_healthy',
            'extravasation_injury',
            'kidney_healthy',
            'kidney_low',
            'kidney_high',
            'liver_healthy',
            'liver_low',
            'liver_high',
            'spleen_healthy',
            'spleen_low',
            'spleen_high',
            #'any_injury'
        ]
        label_a = train_csv.loc[train_csv.patient_id == patient_id, label_columns].values[0].astype('float32')
        # for i in range(4, 13):
        #     if label_a[i] == 0:
        #         label_a[i] = 0.01
        #     else:
        #         label_a[i] = 0.98
        label_t = torch.from_numpy(label_a)
        return img_t, label_t

In [7]:
train_meta = train_series_meta[-3600:].reset_index()
val_meta = train_series_meta[0:-3600].reset_index()

# train_meta = injury_series_meta[-1000:].reset_index()
# val_meta = injury_series_meta[0:-1000].reset_index()

train_ds = CTDataset(meta = train_meta, augmentation=True)
train_dl = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=8)

val_ds = CTDataset(meta = val_meta, augmentation=False)
val_dl = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=8)

In [8]:
class EffNet(nn.Module):
    def __init__(self, ch_out=9):
        super(EffNet, self).__init__()
        #self.conv_in = nn.Conv3d(1, 3, kernel_size=5, padding=2, stride=2)
        self.net = EfficientNetBN("efficientnet-b0", pretrained=False, progress=False, spatial_dims=3, in_channels=1, num_classes=ch_out,)
    def forward(self, x):
        #x = self.conv_in(x)
        #return torch.sigmoid(self.net(x))
        return self.net(x)

In [9]:
import sklearn.metrics
from scipy.special import softmax
from scipy.special import expit

def transform_9class(label_in):
    label_out = [1 - label_in[0],
                   label_in[0],
                    1- label_in[1],
                   label_in[1],
                   (1 - label_in[2]) * (1 - label_in[3]),
                   label_in[2],
                   label_in[3],
                   (1 - label_in[4]) * (1 - label_in[5]),
                    label_in[4],
                   label_in[5],
                   (1 - label_in[6]) * (1 - label_in[7]),
                   label_in[6],
                   label_in[7]]
    return label_out

def transform_13class(label_in):
    label_out = label_in
    return label_out.tolist()

def transform_13class_crossentropy(label_in):
    bowl_label = label_in[0:2]
    extravasation_label = label_in[2:4]
    kidney_label = label_in[4:7]
    liver_label = label_in[7:10]
    spleen_label = label_in[10:13]
    
    label_in[0:2] = softmax(bowl_label)
    label_in[2:4] = softmax(extravasation_label)
    label_in[4:7] = softmax(kidney_label)
    label_in[7:10] = softmax(liver_label)
    label_in[10:13] = softmax(spleen_label)
    
    
    label_out = label_in
    return label_out.tolist()


def loss_metrics(metrics, transform):
    preds = [transform(x) for x in metrics["predict"]]
    targets = [np.round(x).tolist() for x in metrics["label"]]
    targets_any_injury = metrics["label"][:, -1]
    
    loss_list = []
    
    print("F1 score: ", sklearn.metrics.f1_score(targets, np.around(preds), average=None, zero_division=0.0))
    
    for i in range(0, len(preds)):
        predict = preds[i]
        target = targets[i]
        
        label_pred = np.zeros(14)
        label_pred[0] = predict[0] / (predict[0] + predict[1])
        label_pred[1] = predict[1] / (predict[0] + predict[1])
        label_pred[2] = predict[2] / (predict[2] + predict[3])
        label_pred[3] = predict[3] / (predict[2] + predict[3])        
        label_pred[4] = predict[4] / (predict[4] + predict[5] + predict[6])
        label_pred[5] = predict[5] / (predict[4] + predict[5] + predict[6])
        label_pred[6] = predict[6] / (predict[4] + predict[5] + predict[6])
        label_pred[7] = predict[7] / (predict[7] + predict[8] + predict[9])
        label_pred[8] = predict[8] / (predict[7] + predict[8] + predict[9])
        label_pred[9] = predict[9] / (predict[7] + predict[8] + predict[9])
        label_pred[10] = predict[10] / (predict[10] + predict[11] + predict[12])
        label_pred[11] = predict[11] / (predict[10] + predict[11] + predict[12])
        label_pred[12] = predict[12] / (predict[10] + predict[11] + predict[12])
        label_pred[13] = max([1 - label_pred[x] for x in [0, 2, 4, 7, 10]])
        
        targets_any_injury = max([1 - target[x] for x in [0, 2, 4, 7, 10]])
        
        target.append(targets_any_injury)
        label_target = np.array(target)
        
        weight = np.array([1, 2, 1, 6, 1, 2, 4, 1, 2, 4, 1, 2, 4, 6])
        
        loss_list.append(sklearn.metrics.log_loss(
            y_true=label_target,
            y_pred=label_pred,
            sample_weight=weight))
    #print("Weighted Loss: " + np.mean(loss_list))
    
    return np.mean(loss_list)
        
    
    
    #print(np.array(preds).shape)
    

In [10]:
import copy

def loss_fn(outputs, labels):
    bowl_label = labels[:, 0:2]
    bowl_pred = outputs[:, 0:2]
    extravasation_label = labels[:, 2:4]
    extravasation_pred = outputs[:, 2:4]
    kidney_label = labels[:, 4:7]
    kidney_pred = outputs[:, 4:7]
    liver_label = labels[:, 7:10]
    liver_pred = outputs[:, 7:10]
    spleen_label = labels[:, 10:13]
    spleen_pred = outputs[:, 10:13]
    
    #loss_bowl = nn.CrossEntropyLoss(weight=torch.Tensor([1, 30]).to(device))
    #loss_extravasation = nn.CrossEntropyLoss(weight=torch.Tensor([1, 15]).to(device))
    loss_bowl = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([1, 30]).to(device))
    loss_extravasation = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([1, 15]).to(device))
    loss_kidney = nn.CrossEntropyLoss(weight=torch.Tensor([1, 30, 30]).to(device))
    loss_liver = nn.CrossEntropyLoss(weight=torch.Tensor([1, 18, 18]).to(device))
    loss_spleen = nn.CrossEntropyLoss(weight=torch.Tensor([1, 18, 18]).to(device))
    # loss_bowl = nn.BCEWithLogitsLoss()
    # loss_extravasation = nn.BCEWithLogitsLoss()
    # loss_kidney = nn.CrossEntropyLoss()
    # loss_liver = nn.CrossEntropyLoss()
    # loss_spleen = nn.CrossEntropyLoss()
    
    loss = (loss_bowl(bowl_pred, bowl_label) + 
            loss_extravasation(extravasation_pred, extravasation_label) + 
            loss_kidney(kidney_pred, kidney_label) + 
            loss_liver(liver_pred, liver_label) + 
            loss_spleen(spleen_pred, spleen_label))
    return loss

def TrainClassifer(model,trn_dl,val_dl,optimizer, scheduler=None,
                   n_eopchs=20, device='cpu'):
 
    #loss_fn = nn.BCELoss(weight=torch.Tensor([1, 6, 1, 6, 1, 4, 8, 1, 4, 8, 1, 4, 8]).to(device))
    #loss_fn = nn.BCELoss()
    #loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([1, 30, 1, 15, 1, 30, 30, 1, 18, 18, 1, 18, 18]).to(device))
    model.to(device)
    best_model = copy.deepcopy(model)
    bestweight_model = copy.deepcopy(model)
    best_val = 999.0
    best_weightloss = 999.0
    metrics = {'predict': [], 'label' : []}
    PATH_MODEL = 'EffNet3D-tune/normal_13class_posweight_bestloss.pt'
    PATH_MODEL2 = 'EffNet3D-tune/normal_13class_posweight_bestmetric.pt'
    wandb.init(name='EffNet_tune_normal_13class_posweight_patient', 
               project='EffNet3D-tune')

    for epoch in range(1, n_eopchs + 1):
        loss_train = 0.0
        model.train()
        for imgs, labels in tqdm(trn_dl, position=0):
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            #outputs = model(imgs.unsqueeze(1))
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        torch.cuda.empty_cache()
        
        loss_val = 0.0
        correct_val = 0.0
        model.eval()
        
        for imgs, labels in tqdm(val_dl, position=0):
            imgs = imgs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = model(imgs)
                #outputs = model(imgs.unsqueeze(1))
                loss = loss_fn(outputs, labels)
                loss_val += loss.item()
                #outputs = torch.sigmoid(outputs)
                metrics['predict'].extend((outputs.to('cpu').detach().numpy()).tolist())
                metrics['label'].extend((labels.to('cpu').detach().numpy()).tolist())
        
        metrics['predict'] = np.array(metrics['predict'])
        #metrics['predict'] = np.array([[0.5, 0.5, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333]]*len(metrics['label']))
        metrics['label'] = np.array(metrics['label'])
        weighted_loss = loss_metrics(metrics, transform_13class_crossentropy)
        metrics = {'predict': [], 'label' : []}
        
        torch.cuda.empty_cache()
        
        if (weighted_loss) < best_weightloss:
            best_weightloss = weighted_loss
            torch.save(model.state_dict(), 'model_tmp.pt')
            bestweight_model.load_state_dict(torch.load('model_tmp.pt'))
            
        if loss_val / len(val_dl) < best_val:
            best_val = loss_val / len(val_dl)
            torch.save(model.state_dict(), 'model_tmp.pt')
            best_model.load_state_dict(torch.load('model_tmp.pt'))
            
            
        if scheduler != None:
            scheduler.step()

        print('{} Eopch {}, Training Loss {}, Val Loss {}, Weighted Loss {}'.format(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()),
                                                                  epoch, loss_train / len(trn_dl), loss_val / len(val_dl),
                                                                                     weighted_loss))
        
        
        
        wandb.log({'training loss': loss_train / len(trn_dl),
                  'val loss': loss_val / len(val_dl),
                  'weighted loss': weighted_loss})
    torch.save(best_model.state_dict(), PATH_MODEL)
    torch.save(bestweight_model.state_dict(), PATH_MODEL2)
    print('Finish training: best_val:{}, best_weighted loss:{}'.format(best_val, best_weightloss))
    wandb.finish()

In [11]:
net = EffNet(ch_out=13).to(device)

optimizer = optim.AdamW(net.parameters(), lr=1e-4)
TrainClassifer(model=net,trn_dl=train_dl,val_dl=val_dl,optimizer=optimizer, 
               scheduler=None, n_eopchs=30, device=device)

wandb: Currently logged in as: northm (rsna2023). Use `wandb login --relogin` to force relogin


100%|██████████| 278/278 [01:00<00:00,  4.60it/s]


F1 score:  [0.98677611 0.         0.96603071 0.         0.6954933  0.04878049
 0.         0.62508295 0.16536661 0.         0.         0.13376483
 0.        ]
2023-09-19 03:15:21 Eopch 1, Training Loss 14.546101359261407, Val Loss 26.034089627883418, Weighted Loss 1.1473969342345165


100%|██████████| 278/278 [01:03<00:00,  4.40it/s]


F1 score:  [0.98677611 0.         0.96603071 0.         0.         0.08274232
 0.         0.00603015 0.14970563 0.         0.         0.
 0.06361829]
2023-09-19 03:21:17 Eopch 2, Training Loss 12.049710024992624, Val Loss 14.491142020808708, Weighted Loss 0.7518165801490556


...

100%|██████████| 278/278 [01:02<00:00,  4.43it/s]


F1 score:  [0.98446069 0.         0.94561069 0.0952381  0.57489879 0.07453416
 0.15668203 0.56413891 0.20364742 0.         0.64982935 0.13513514
 0.29319372]
2023-09-19 06:00:32 Eopch 29, Training Loss 8.381470173199972, Val Loss 10.389443567330888, Weighted Loss 0.45367130317604326


100%|██████████| 278/278 [01:01<00:00,  4.52it/s]


F1 score:  [0.98631387 0.         0.96554935 0.         0.76860465 0.05333333
 0.21818182 0.70728643 0.20664207 0.         0.73235855 0.17391304
 0.24761905]
2023-09-19 06:06:25 Eopch 30, Training Loss 8.380880272123548, Val Loss 10.831680521047373, Weighted Loss 0.39131052132088895
Finish training: best_val:10.173356005399347, best_weighted loss:0.373864370180773


training loss,█▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁
val loss,█▃▂▂▁▁▁▁▂▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
weighted loss,█▄▄▂▂▂▂▂▂▂▂▂▂▁▃▁▂▃▂▁▂▂▂▂▁▂▁▁▂▁
training loss,8.38088
val loss,10.83168
weighted loss,0.39131


In [12]:
net = EffNet(ch_out=13).to(device)
net.load_state_dict(torch.load('model_tmp.pt'))
net.eval()
imgs, labels = next(iter(val_dl))
print(net(imgs.to(device)))
print(labels)

metatensor([[ 5.4777, -2.1630,  3.7284, -1.0469,  1.3183,  0.7446, -1.0790,  1.4052,
          1.0009, -1.1236,  1.8384,  0.3309, -1.9226],
        [ 5.5490, -2.3044,  3.7017, -0.9131,  1.0832,  0.7730, -0.9335,  1.2717,
          1.0988, -1.1344,  1.8401,  0.3495, -1.9262],
        [ 4.6217, -1.2320,  3.2242, -0.6879,  1.6444,  0.5884, -1.1596,  1.6119,
          0.8541, -1.1229,  1.5023,  0.3066, -1.4751],
        [ 3.8422, -1.2337,  2.6608, -0.4747,  1.7175,  0.3330, -0.9043,  1.1602,
          0.5149, -0.4375,  0.4147, -0.0238,  0.0256]], device='cuda:0',
       grad_fn=<AliasBackward0>)
tensor([[1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
        [1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.]])
